# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [1]:
import redis

In [2]:
r = redis.Redis(host='localhost', port=6379)

In [3]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [4]:
r.set('greetings', 'hello world')
retrieved_value = r.get('greetings')
print(retrieved_value)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [5]:
r.set('connections', 0)
try:
    print(r.incr((r.get('greetings'))))
except redis.exceptions.ResponseError as e:
    print("Error:", e)


1


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [9]:
# create a list of items using lpush 
r.lpush('colors', 'red', 'blue', 'black')


[b'black', b'blue', b'red']


**Q** : Display the second element of the list with `LINDEX`

In [10]:
# the secoond element in the list with index 
print(r.lindex('colors', 1))

b'blue'


**Q** : Display the full list with `LRANGE`

In [12]:
print(r.lrange('colors', 0, -1))

[b'black', b'blue', b'red']


### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [19]:
r.sadd("nosql", "redis", "mongodb", "orientdb")

0

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [20]:
r.sismember("nosql", "redis")

1

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [21]:
# add a new element to the set with smembe
r.smembers("hbase")
r.sadd("nosql", "hbase")
print(r.smembers("nosql"))

r.sadd("nosql", "redis")
print(r.smembers("nosql"))

{b'redis', b'hbase', b'mongodb', b'orientdb'}
{b'redis', b'hbase', b'mongodb', b'orientdb'}


### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [22]:
r.zadd("top14", {
    "Agen": 10,
    "Bordeaux": 33,
    "Brive": 32,
    "Castres": 29,
    "Clermont": 38,
    "Grenoble": 24,
    "La Rochelle": 26,
    "Montpellier": 32,
    "Oyonnax": 14,
    "Pau": 20,
    "Racing": 40,
    "Stade Français": 22,
    "Toulon": 36,
    "Toulouse": 36
})

14

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [23]:
score_toulon = r.zscore("top14", "Toulon")
rank_toulon = r.zrank("top14", "Toulon")
print("Toulon score:", score_toulon)
print("Toulon rank:", rank_toulon)

Toulon score: 36.0
Toulon rank: 10


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [24]:
rank_toulon_desc = r.zrevrank("top14", "Toulon")
print("Toulon rank (desc):", rank_toulon_desc)

Toulon rank (desc): 3


**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [25]:
# Display the 3 best teams (using ZREVRANGE)
three_best = r.zrevrange("top14", 0, 2, withscores=True)
print("3 best teams:", three_best)

# Display teams with more than 35 points (using ZRANGEBYSCORE)
teams_more_than_35 = r.zrangebyscore("top14", 35, "+inf", withscores=True)
print("Teams with more than 35 points:", teams_more_than_35)

3 best teams: [(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]
Teams with more than 35 points: [(b'Toulon', 36.0), (b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]


### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [30]:
# create dictionary with HMSET
r.hmset("user:1", {
    "id": 1,
    "name": "Jean",
    "age": 22
})

# display the user with HGETALL
user = r.hgetall("user:1")
print("User:", user)

User: {b'id': b'1', b'name': b'Jean', b'age': b'22', b'city': b'Lyon'}


C:\Users\lcisse\AppData\Local\Temp\ipykernel_11896\2660287293.py:2: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset("user:1", {


**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [31]:
# add city lyon to the user with HSET
r.hset("user:1", "city", "Lyon")

# rename jean to paul with HSET
r.hset("user:1", "name", "Paul")

0

In [32]:
# display the user with HGETALL
user = r.hgetall("user:1")
print(user)

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}


## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [35]:
# insert a put request in the cache 
r.set("PUT/http://my-api.fr/user/10", '{"id": 10, "name": "Jean"}')

True

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [38]:
# Insert a get request in the cache on the same key with 2 different values
r.set("GET/http://my-api.fr/user?size=2", '[{"id": 10, "name": "Jean"}, {"id": 11, "name": "Claire"}]')

True

**Q** : Create a set of cache request keys.

In [39]:
# Create a set of cache request keys 
r.sadd("cache_keys", "PUT/http://my-api.fr/user/10", "GET/http://my-api.fr/user?size=2")

2

**Q** : Check if the GET request on http: //http://my-api.fr/user

In [40]:
# Check if the GET request on http: //http://my-api.fr/user
# with the size parameter of 2 is in the cache_keys set
print(r.sismember("cache_keys", "GET/http://my-api.fr/user?size=2"))
print(r.sismember("cache_keys", "GET/http://my-api.fr/user?size=3"))

1
0


**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [41]:
# delete the put request on http://my-api.fr/user/10 from the cache_keys 
r.srem("cache_keys", "PUT/http://my-api.fr/user/10")

1

## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [ ]:
!redis-cli flushall